In [1]:
import RL_samp
from RL_samp.header import *
from RL_samp.utils import *
from RL_samp.replay_buffer import *
from RL_samp.models import poly_net, val_net
from RL_samp.reconstructors import sigpy_solver
from RL_samp.policies import DQN
from RL_samp.trainers import DeepQL_trainer

from importlib import reload

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
/opt/anaconda/envs/pyenv/lib/python3.9/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


In [2]:
datapath = '/mnt/shared_a/OCMR/OCMR_fully_sampled_images/'
ncfiles = list([])
for file in os.listdir(datapath):
    if file.endswith(".pt"):
        ncfiles.append(file)

In [ ]:
loader.reset()
iterMax = 2000
for ind in range(iterMax):
    loader.test()

In [3]:
### Parameter settings

## image parameters
heg = 192
wid = 144

## reconstructor parameters
max_iter = 50
L = 5e-3
solver = 'ADMM'

## trainer parameters
discount    = .5
memory_len  = 20
t_backtrack = 3
base        = 5
budget      = 13
episodes    = 1
save_freq   = 10
batch_size  = 2
ngpu        = 1
lr          = 1e-3
eps         = 1e-3
double_q    = False

In [ ]:
reload(RL_samp)
import RL_samp
import RL_samp.trainers
from RL_samp.trainers import DeepQL_trainer


In [5]:
loader  = ocmrLoader(ncfiles,batch_size=1)
memory  = ReplayMemory(capacity=memory_len,
                       curr_obs_shape=(t_backtrack,heg,wid),
                       mask_shape=(wid),
                       next_obs_shape=(1,heg,wid),
                       batch_size=batch_size,
                       burn_in=batch_size)
model   = poly_net(samp_dim=wid)
policy  = DQN(model,memory,max_iter=max_iter,ngpu=ngpu,gamma=discount,lr=lr,double_q_mode=double_q)
trainer = DeepQL_trainer(loader,policy,episodes=episodes,
                         eps=eps,
                         base=base,budget=budget,
                         ngpu=ngpu)
trainer.train()

current file: fs_0070_1_5T.pt
Dimension of the current data file: t_ubd 19, slice_ubd 12, rep_ubd 1
episode [0/1]
step: 1, burn in, mask sum: 6.0
step: 2, loss: 0.0006, RL reward: 0.0000, Rand reward: 0.0005 
 mask sum: 7.0
step: 3, loss: 0.0009, RL reward: -0.0009, Rand reward: 0.0002 
 mask sum: 8.0
step: 4, loss: 0.0146, RL reward: 0.0006, Rand reward: -0.0009 
 mask sum: 9.0
step: 5, loss: 0.0020, RL reward: 0.0477, Rand reward: 0.0086 
 mask sum: 10.0
step: 6, loss: 0.0003, RL reward: 0.0041, Rand reward: 0.0041 
 mask sum: 11.0
step: 7, loss: 0.0012, RL reward: 0.0104, Rand reward: 0.0010 
 mask sum: 12.0
step: 8, loss: 0.0001, RL reward: 0.0014, Rand reward: -0.0007 
 mask sum: 13.0
step: 9, loss: 0.0002, RL reward: 0.0002, Rand reward: 0.0004 
 mask sum: 14.0
step: 10, loss: 0.0005, RL reward: 0.0002, Rand reward: -0.0002 
 mask sum: 15.0
At step 10, training history saved as /home/huangz78/rl_samp/DQN_hist.pt
step: 11, loss: 0.0003, RL reward: 0.0248, Rand reward: 0.0025 
 mas

KeyboardInterrupt: 

In [ ]:
p_net   = poly_net(samp_dim=wid,softmax=True)
v_net   = val_net()
trainer = AC1_trainer(loader, polynet=p_net, valnet=v_net)
trainer.run()

In [ ]:
import dqn
reload(dqn)
from dqn import DQN

import replay_buffer
reload(replay_buffer)
from replay_buffer import *

import utils
reload(utils)
from utils import *

In [ ]:
trainer.train()